In [2]:
#global variables 

suits = ['spade','heart','club', 'diamond']
suit_codes = {'spade':'\u2660','heart':'\u2661', 'club':'\u2663', 'diamond':'\u2662'}
suit_values = {'spade':4, 'heart':3, 'club':2, 'diamond':1}
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_points = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':10, 'Q':10, 'K':10, 'A':11}
rank_values = {"2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "10":10,"J":11, "Q":12, "K":13, "A":14}
print(suit_codes.values())


dict_values(['♠', '♡', '♣', '♢'])


In [3]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.point = rank_points[rank]
        self.strength = rank_values[rank]*10 + suit_values[suit]
    def __str__(self):
        return suit_codes[self.suit] + self.rank
               
    def __gt__(self, other):
        return self.strength > other.strength
        
    def __eq__(self, other):
        return self.strength == other.strength

    def __ge__(self, other):
        return self > other or self == other
    
    def __lt__(self, other):
        return not self >= other
    
    def __le__(self, other):
        return not self > other
    
    def __cmp__(self, other):
        if self > other:
            return 1
        elif self == other:
            return 0
        else:
            return -1 


In [4]:
import random
class Deck:
    
    def __init__(self):
        self.cards =[Card(suit, rank) for suit in suits for rank in ranks]
        self.__shuffle()
        
    def __shuffle(self):
        random.shuffle(self.cards)
    
    def __str__(self):
        deck_str = [card.__str__() for card in self.cards]
        return '\n'.join(deck_str)
    
    def deal(self):
        return self.cards.pop()

In [5]:
class Hands:
    
    def __init__(self):
        self.cards = []
        self.points = 0
        self.aces = 0
       
    def add_card(self, card):
        self.cards.append(card)
        self.points += card.point
        if card.rank == 'A':
            self.aces += 1
        self.adjust_ace_point()
    
    def rand_card(self, num=5):
        deck = Deck()
        self.cards = deck.cards[0:num]
    
    def adjust_ace_point(self):
        while self.points > 21 and self.aces:
            self.points -= 10
            self.aces -= 1    
    
    def check_flush(self):
        return len(set([card.suit for card in self.cards])) == 1
    
    def check_straight_flush(self):
        return self.check_flush() and self.check_straight()

    def check_four_of_a_kind(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
        return  sorted(list({rank:ranks.count(rank) for rank in ranks}.values())) == [1,4]

    def check_full_house(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
        return  sorted(list({rank:ranks.count(rank) for rank in ranks}.values())) == [2,3]

    def check_straight(self):
        values = [rank_values[card.rank] for card in self.cards]
        values_count = sorted(list({value:values.count(value) for value in values}.values()))
        value_range = max(values) - min(values)
        if len(set(values_count)) == 1 and (value_range==4):
            return True
        else:
            #check straight with low Ace
            return  set(values) == set([14, 2, 3, 4, 5])

    def check_three_of_a_kind(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
        return  sorted(list({rank:ranks.count(rank) for rank in ranks}.values())) == [1,1,3]

    def check_two_pairs(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
        return  sorted(list({rank:ranks.count(rank) for rank in ranks}.values())) == [1,2,2]

    def check_one_pairs(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
        return  sorted(list({rank:ranks.count(rank) for rank in ranks}.values())) == [1,1,1,2]
    
    def check_high_card(self):
        return True
    
    
    def evaluate_hand(self):
        ranks = [card.rank for card in self.cards]
        rank_count = {rank:ranks.count(rank) for rank in ranks}
       
        if self.check_straight_flush():
            if set(ranks) == set(['A', '2','3','4','5']):
                return 9000
            return 9000 + max(self.cards).strength
        
        elif self.check_four_of_a_kind():
            same_rank = [card for card in self.cards if rank_count[card.rank] == 4]
            return 8000 + max(same_rank).strength
        
        elif self.check_full_house():
            same_rank = [card for card in self.cards if rank_count[card.rank] == 3]
            return 7000 + max(same_rank).strength
        
        elif self.check_flush():
            return 6000 + max(self.cards).strength
        
        elif self.check_straight():
            if set(ranks) == set(['A', '2','3','4','5']):
                return 5000
            return 5000 + max(self.cards).strength
        
        elif self.check_three_of_a_kind():
            same_rank = [card for card in self.cards if rank_count[card.rank] == 3]
            return 4000 + max(same_rank).strength
        
        elif self.check_two_pairs():
            same_rank = [card for card in self.cards if rank_count[card.rank] == 2]
            return 3000 + max(same_rank).strength
        
        elif self.check_one_pairs():
            same_rank = [card for card in self.cards if rank_count[card.rank] == 2]
            return 2000 + max(same_rank).strength
        
        elif self.check_high_card():
            return 1000 + max(self.cards).strength
    
    def __str__(self, print_all = True):
        hand_str = [card.__str__() for card in self.cards]
        if not print_all:
            hand_str[0] = "<Hole Card>"
        return '\n'.join(hand_str)
    
    def __gt__(self, other):
        
        return self.evaluate_hand() > other.evaluate_hand()
        
    def __eq__(self, other):
        return self.evaluate_hand() == other.evaluate_hand()

    def __ge__(self, other):
        return self > other or self == other
    
    def __lt__(self, other):
        return not self >= other
    
    def __le__(self, other):
        return not self > other
    
    def __cmp__(self, other):
        if self > other:
            return 1
        elif self == other:
            return 0
        else:
            return -1 

In [6]:
class Chips:
    
    def __init__(self, total=100, ante=0):
        self.total = total
        self.ante = ante
    
    def win_bet(self):
        self.total += self.ante
    
    def lose_bet(self):
        self.total -= self.ante



In [7]:
class Person:
    
    def __init__(self, hands=Hands(), chips=Chips()):
        self.hands = hands
        self.chips = chips
    
    def init_card(self, deck, num=2):
        self.hands = Hands()
        for i in range(num):
            self.hands.add_card(deck.deal())
    
    def take_ante(self, ante):
        self.chips.ante = ante
    
    def hit(self, deck):
        self.hands.add_card(deck.deal())


In [8]:
class BlackjackGame:
    
    def __init__(self):
        self.stand = False
        self.banker = Person()
        self.player = Person()
        self.deck = Deck()
        
    def start(self):
        self.stand = False
        self.banker.init_card(self.deck)
        self.player.init_card(self.deck)
    
    def show_some(self):
        print("Banker's hand:")
        print(self.banker.hands.__str__(False))
        print("Player's hand:")
        print(self.player.hands)
    
    def show_all(self):
        print("Banker's hands:")
        print(self.banker.hands)
        print("Banker's Hands = ",self.banker.hands.points)
        print("Player's hands:")
        print(self.player.hands)
        print("Player's Hands = ", self.player.hands.points)
    
    def run(self):
        print("Welcome to the Blackjack Game:\n")
        ante = int(input("Please input your ante: \n"))
        self.player.take_ante(ante)
        
        self.start()
        20
        self.show_some()
        ##player to take hits or stand#
        
        while not self.stand:
            x = input("Would you like to Hit or Stand? Enter 'h' or 's'")
            if x[0].lower() == 'h':
                self.player.hit(self.deck)
                
            elif x[0].lower() == 's':
                print("Player stands. Dealer is playing.")
                self.stand = True
            
            if self.player.hands.points > 21:
                print("Player burst, Dealer wins!")
                self.player.chips.lose_bet()
                self.show_all()
                break
            self.show_some()
        
            
        # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
        while self.banker.hands.points < 17:
            self.banker.hit(self.deck)
            
        # Show all cards
        self.show_all()
                
            
        if self.banker.hands.points > 21 or self.banker.hands.points < self.player.hands.points:
            print("Player wins!")
            self.player.chips.win_bet()
        
        elif self.banker.hands.points > self.player.hands.points:
            print("Banker wins!")
            self.player.chips.lose_bet()
                       
        else:
            print("Banker and Player tie! It's a push.")
        
        # Inform Player of their chips total
        print("\nPlayers winnings stand at ", self.player.chips.total)
        
    def loop_run(self):
        while True:
            self.run()
            # Ask to play again
            new_game = input("would you like to play again? Enter 'y' or 'n'").strip()
            if new_game[0].lower() == 'y':
                self.stand = False
                continue
            else:
                print('Thanks for playing! ')
                break
blackjack = BlackjackGame()
blackjack.loop_run()

Welcome to the Blackjack Game:

Please input your ante: 
20
Banker's hand:
<Hole Card>
♢2
Player's hand:
♠7
♣7
Would you like to Hit or Stand? Enter 'h' or 's'h
Banker's hand:
<Hole Card>
♢2
Player's hand:
♠7
♣7
♠6
Would you like to Hit or Stand? Enter 'h' or 's's
Player stands. Dealer is playing.
Banker's hand:
<Hole Card>
♢2
Player's hand:
♠7
♣7
♠6
Banker's hands:
♡J
♢2
♡3
♢5
Banker's Hands =  20
Player's hands:
♠7
♣7
♠6
Player's Hands =  20
Banker and Player tie! It's a push.

Players winnings stand at  100
would you like to play again? Enter 'y' or 'n'100
Thanks for playing! 


In [11]:
class FiveCardStud:
    
    def __init__(self):
        self.round_end = 0
        self.banker = Person()
        self.player = Person()
        self.deck = Deck()
        
    def init(self):
        self.playing = True
        self.banker.init_card(self.deck)
        self.player.init_card(self.deck)
    
    def show_some(self):
        print("Banker's hand:")
        print(self.banker.hands.__str__(False))
        print("Player's hand:")
        print(self.player.hands)
    
    def show_all(self):
        print("Banker's hands:")
        print(self.banker.hands)
        print("Player's hands:")
        print(self.player.hands)
    
    def run(self):
        print("Welcome to the Blackjack Game:\n")
        ante = int(input("Please input your ante: \n"))
        self.player.take_ante(ante)
        
        self.init()
        
        self.show_some()
        ##player to take hits or stand#
        
        while self.round_end < 3:
            x = input("Would you like to call or not? Enter 'y' or 'n'")
            if x[0].lower() == 'y':
                self.player.chips.ante += ante
                self.player.hit(self.deck)
                self.banker.hit(self.deck)
            elif x[0].lower() == 'n':
                print("Banker wins!")
                self.player.chips.lose_bet()
                break
            self.round_end += 1
            self.show_some()
            
        self.show_all()
        if self.banker.hands > self.player.hands:
            print("Banker wins!")
            self.player.chips.lose_bet()
        
        elif self.banker.hands < self.player.hands:
            print("Banker wins!")
            self.player.chips.win_bet()
        
        else:
            print("Banker and Player tie! It's a push.")
        
        # Inform Player of their chips total
        print("\nPlayers winnings stand at ", self.player.chips.total)
    def loop_run(self):
        while True:
            self.run()
            # Ask to play again
            new_game = input("would you like to play again? Enter 'y' or 'n'").strip()
            if new_game[0].lower() == 'y':
                self.round_end = 0
                continue
            else:
                print('Thanks for playing! ')
                break
showhand = FiveCardStud()
showhand.loop_run()
            
            
        

Welcome to the Blackjack Game:

Please input your ante: 
20
Banker's hand:
<Hole Card>
♡5
Player's hand:
♢5
♢7
Would you like to call or not? Enter 'y' or 'n'y
Banker's hand:
<Hole Card>
♡5
♢A
Player's hand:
♢5
♢7
♢J
Would you like to call or not? Enter 'y' or 'n'y
Banker's hand:
<Hole Card>
♡5
♢A
♠7
Player's hand:
♢5
♢7
♢J
♣5
Would you like to call or not? Enter 'y' or 'n'y
Banker's hand:
<Hole Card>
♡5
♢A
♠7
♣4
Player's hand:
♢5
♢7
♢J
♣5
♢9
Banker's hands:
♢10
♡5
♢A
♠7
♣4
Player's hands:
♢5
♢7
♢J
♣5
♢9
Banker wins!

Players winnings stand at  100
would you like to play again? Enter 'y' or 'n'n
Thanks for playing! 
